In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

# Assumptions

In [2]:
# Gas Truck Aassumptions
g_tube_unit = 574000 #$/module
g_trailer = 60000 #$/trailer
g_variable_o_m = 0.01 # for Non-Fuel
g_fixed_o_m = 0.04
crf_tube_trailer = 0.12
g_truck_capacity = 796 #kg/truck
g_pmax = 500  # 500 atm
g_pmin = 50 # atm
g_net_delivery = 661 #kg/truck key issue
g_load_unload_time = 3 #hours/ trip

In [3]:
# Liquid Truck Assumptions

l_tank_unit = 890000 #$/module
l_trailer = 60000 #$/trailer
l_variable_o_m = 0.01 # for Non-Fuel
l_fixed_o_m = 0.04
crf_tank_trailer = 0.12
l_truck_boil_off_rate = 0.30 # %/day
l_truck_capacity = 4000 # kg/truck
l_load_unload_time = 5.5 #hr/ trip

In [4]:
# General Truck Assumptions
cab_cost = 350000 #$/ cab
crf_cab = 0.26
fuel_economy = 7.11 # miles/kg of hydrogen
average_speed = 40 #km/hr
truck_availability = 24 #hr/day
hour_per_driver = 9.8 #hr/driver
driver_wage_benefits = 24.74 #$/hr
truck_cf = 0.8 # truck capacity factor
driving_circuity_factor = 0
fuel_price = 9 # $/kg

In [5]:
# Liquefaction Assumptions

l_base_plant_size = 20000 #kg/day
l_cost_base_plant = 62000000 # $
scale_factor_liquefier = 0.80
CRF_liquefier = 0.1
l_energy_use = 10 # kWh/kg
l_electricity_cost = 0.080 # $/kWh
l_o_and_m_cost = 0.05

# Compression

c_energy_use = 3.41286114500405 # kWh/d
c_overcapacity = 0.23
c_base_size = 20 #kW
c_base_cost = 245353
c_scale_factor = 0.46
c_crf = 0.13
c_o_and_m_cost = 0.04
c_electricity_cost = 0.08 # $/kWh

In [129]:
# Electricity 

electricity_cost = 0.08
ca_primary_energy = 2.5 #ratio
elec_CO2_emissions = 369 # grams/ kWh

annual_plant_cf = 1
h_fuel_ci = 4840 # kg of CO2e/kg H2

# Functions to Run the Model

In [7]:
# Function to see how many stations will be created

def number_of_stations(delivery_requirement, station_size, system_utilization):
    number_of_stations = delivery_requirement/station_size/system_utilization
    return number_of_stations

def idealized_city_radius_truck_delivery_distance(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance):
    station_count = number_of_stations(delivery_requirement, station_size, system_utilization)
    icm_factor = 1.4234
    truck_distance = icm_factor * station_count # based on the number of stations
    average_delivery_distance = city_radius * truck_distance/ station_count + transmission_distance
    return average_delivery_distance

In [58]:
# Function to see the design size of the compressor

def design_compressor_truck(delivery_requirement, system_utilization):
    # Only need to change the size of the compressors because trucks are more variable and can be based on the actual demand
    
    design_capacity = 0 # to initialize the design capacity
    
    if delivery_requirement/ system_utilization < delivery_requirement:
        design_capacity = delivery_requirement
    else: 
        design_capacity = delivery_requirement/ system_utilization

    design_annual_production = design_capacity * annual_plant_cf * 365 # multiplying since there is 365 days in a year
    total_compressor_energy_use = c_energy_use * design_capacity * annual_plant_cf
    

    actual_c_energy_use = c_energy_use * delivery_requirement * annual_plant_cf
    design_compressor_size = total_compressor_energy_use/ 24 * (1+c_overcapacity)
    
    return design_compressor_size, actual_c_energy_use
    
def design_liquefier_truck(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance):
    # Only need to change the size of the compressors because trucks are more variable and can be based on the actual demand
    
    design_capacity = 0 # to initialize the design capacity
    
    if delivery_requirement/ system_utilization < delivery_requirement:
        design_capacity = delivery_requirement
    else: 
        design_capacity = delivery_requirement/ system_utilization
    
    annual_production = delivery_requirement * 365 * annual_plant_cf

    average_delivery_distance = idealized_city_radius_truck_delivery_distance(delivery_requirement, station_size, 
                                                                              system_utilization, city_radius, 
                                                                              transmission_distance)
    
    driving_time_per_trip = math.floor(average_delivery_distance * 2/ average_speed*10)/10
    
    delivered_product = annual_production * math.e**(-(driving_time_per_trip/2)*l_truck_boil_off_rate/100) # Might need to change depending on if we include the leakage rate or not
        
    liquefier_plant_size = design_capacity * annual_plant_cf
    actual_l_energy_use = l_energy_use * delivered_product * annual_plant_cf
    return liquefier_plant_size, actual_l_energy_use, delivered_product

## Gas Trucks Costs Calculations

In [51]:
def gas_truck_capital_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance):
    average_delivery_distance = idealized_city_radius_truck_delivery_distance(delivery_requirement, station_size, 
                                                                              system_utilization, city_radius, 
                                                                              transmission_distance)
    annual_production = delivery_requirement * 365 * annual_plant_cf
    
    driving_time_per_trip = math.floor(average_delivery_distance * 2/ average_speed*10)/10
    gas_t_availability = truck_availability * truck_cf * 365

    trips_per_year = annual_production/ g_net_delivery
    time_for_each_trip = average_delivery_distance * 2 / average_speed
    
    total_driving_time = trips_per_year * time_for_each_trip
    total_loading_unloading_time = trips_per_year * g_load_unload_time
    total_delivery_time = total_driving_time + total_loading_unloading_time

    truck_requirement = math.ceil(total_delivery_time/ gas_t_availability)

# Need double the amount of tube trailers to fulfill the delivery 
    
    
    tube_trailer_requirement = 2 * truck_requirement
    tubes_undercarriage_requirement_costs = tube_trailer_requirement * (g_tube_unit + g_trailer)
    truck_requirement_costs = truck_requirement * cab_cost 
    total_truck_trailer_capital_cost =  tubes_undercarriage_requirement_costs + truck_requirement_costs

    # Now have to calculate the compressor capital costs

    design_compressor_size, actual_c_energy_use  = design_compressor_truck(delivery_requirement, system_utilization)

    compressor_capital_costs = c_base_cost * (design_compressor_size/ c_base_size)**c_scale_factor
    total_capital_costs = total_truck_trailer_capital_cost + compressor_capital_costs
    
    return total_truck_trailer_capital_cost, compressor_capital_costs, total_capital_costs, tubes_undercarriage_requirement_costs, truck_requirement_costs, total_truck_trailer_capital_cost

In [52]:
def gas_truck_operating_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance):
    
    average_delivery_distance = idealized_city_radius_truck_delivery_distance(delivery_requirement, station_size, 
                                                                              system_utilization, city_radius, 
                                                                              transmission_distance)
    annual_production = delivery_requirement * 365 * annual_plant_cf
    driver_time = truck_cf * 365 * hour_per_driver # hours/ year

    trips_per_year = annual_production/ g_net_delivery
    time_for_each_trip = average_delivery_distance * 2 / average_speed
    
    total_driving_time = trips_per_year * time_for_each_trip
    total_loading_unloading_time = trips_per_year * g_load_unload_time
    total_delivery_time = total_driving_time + total_loading_unloading_time
    
    drivers_required = math.ceil(total_delivery_time/ driver_time)
    
    labor_costs = driver_wage_benefits  * driver_time * drivers_required
    total_distance = trips_per_year * average_delivery_distance * 2 # to account for going there and back
    fuel_usage = math.floor(total_distance/1.60934/fuel_economy*1000)/1000
    fuel_costs = fuel_usage * fuel_price

    design_compressor_size, actual_c_energy_use = design_compressor_truck(delivery_requirement, system_utilization)
    compressor_electricity_costs = actual_c_energy_use*c_electricity_cost*365

    total_truck_trailer_capital_cost, compressor_capital_costs, total_capital_costs, tubes_undercarriage_requirement_costs, truck_requirement_costs, total_truck_trailer_capital_costs = gas_truck_capital_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance)
    variable_non_fuel_o_m = g_variable_o_m * total_truck_trailer_capital_cost # %/yr of capital

    total_variable_operating_costs = labor_costs + fuel_costs + compressor_electricity_costs + variable_non_fuel_o_m


    
    truck_fixed_o_m = total_truck_trailer_capital_cost * g_fixed_o_m
    compressor_o_m_costs = c_o_and_m_cost * compressor_capital_costs
    truck_annual_capital = tubes_undercarriage_requirement_costs * crf_tube_trailer + truck_requirement_costs * crf_cab
    compressor_annual_capital = compressor_capital_costs * c_crf

    total_operating_costs = total_variable_operating_costs +  truck_fixed_o_m  + compressor_o_m_costs + truck_annual_capital + compressor_annual_capital 
    
    storage_costs = 0.072 #$/kg
    per_unit_costs = total_operating_costs/(delivery_requirement * annual_plant_cf * 365)+ storage_costs

    fuel_co2_emissions = fuel_usage * h_fuel_ci/ 1000000 * 1000000 / (delivery_requirement * annual_plant_cf * 365)
    compression_co2_emissions = c_energy_use * elec_CO2_emissions
    total_emissions_per_unit =  fuel_co2_emissions + compression_co2_emissions
    
    return total_operating_costs, per_unit_costs,total_emissions_per_unit

## Liquid Trucks Costs Calculations
    

In [61]:
def liquid_truck_capital_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance):
    average_delivery_distance = idealized_city_radius_truck_delivery_distance(delivery_requirement, station_size, 
                                                                              system_utilization, city_radius, 
                                                                              transmission_distance)
    annual_production = delivery_requirement * 365 * annual_plant_cf
    
    driving_time_per_trip = math.floor(average_delivery_distance * 2/ average_speed*10)/10
    liquid_t_availability = truck_availability * truck_cf * 365

    trips_per_year = annual_production/ l_truck_capacity
    time_for_each_trip = average_delivery_distance * 2 / average_speed
    
    total_driving_time = trips_per_year * time_for_each_trip
    total_loading_unloading_time = trips_per_year * l_load_unload_time
    total_delivery_time = total_driving_time + total_loading_unloading_time

    truck_requirement = math.ceil(total_delivery_time/ liquid_t_availability)
    
    tank_undercarriage_costs = 1 * truck_requirement * (l_tank_unit + l_trailer) # Change the 1 if you need more than one truck
    truck_requirement_costs = truck_requirement * cab_cost 
    total_truck_trailer_capital_cost =  tank_undercarriage_costs + truck_requirement_costs

    # Now have to calculate the liquefier capital costs

    liquefier_plant_size, actual_l_energy_use, delivered_product = design_liquefier_truck(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance)

    if liquefier_plant_size < l_base_plant_size:
        liquefier_capital_costs = l_cost_base_plant * (liquefier_plant_size/ l_base_plant_size)
    else: 
        liquefier_capital_costs = l_cost_base_plant * (liquefier_plant_size/ l_base_plant_size)**scale_factor_liquefier
    
    total_capital_costs = total_truck_trailer_capital_cost + liquefier_capital_costs
    
    return total_truck_trailer_capital_cost, liquefier_capital_costs, total_capital_costs, tank_undercarriage_costs, truck_requirement_costs, total_truck_trailer_capital_cost

In [76]:
def liquid_truck_operating_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance):
    
    average_delivery_distance = idealized_city_radius_truck_delivery_distance(delivery_requirement, station_size, 
                                                                              system_utilization, city_radius, 
                                                                              transmission_distance)
    annual_production = delivery_requirement * 365 * annual_plant_cf
    driver_time = truck_cf * 365 * hour_per_driver # hours/ year

    trips_per_year = annual_production/ l_truck_capacity
    time_for_each_trip = average_delivery_distance * 2 / average_speed
    
    total_driving_time = trips_per_year * time_for_each_trip
    total_loading_unloading_time = trips_per_year * l_load_unload_time
    total_delivery_time = total_driving_time + total_loading_unloading_time
    
    drivers_required = math.ceil(total_delivery_time/ driver_time)
    
    labor_costs = driver_wage_benefits  * driver_time * drivers_required
    total_distance = trips_per_year * average_delivery_distance * 2 # to account for going there and back
    fuel_usage = math.floor(total_distance/1.60934/fuel_economy*1000)/1000
    fuel_costs = fuel_usage * fuel_price

    design_liquefier_size, actual_l_energy_use, delivered_product = design_liquefier_truck(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance)
    liquefier_electricity_costs = actual_l_energy_use * l_electricity_cost

    total_truck_trailer_capital_cost, liquefier_capital_costs, total_capital_costs, tank_undercarriage_costs, truck_requirement_costs, total_truck_trailer_capital_cost = liquid_truck_capital_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance)
    variable_non_fuel_o_m = l_variable_o_m * total_truck_trailer_capital_cost # %/yr of capital

    total_variable_operating_costs = labor_costs + fuel_costs + liquefier_electricity_costs + variable_non_fuel_o_m


    
    truck_fixed_o_m = total_truck_trailer_capital_cost * l_fixed_o_m
    liquefier_o_m_costs = l_o_and_m_cost * liquefier_capital_costs
    truck_annual_capital = tank_undercarriage_costs * crf_tank_trailer + truck_requirement_costs * crf_cab
    liquefier_annual_capital = liquefier_capital_costs * CRF_liquefier

    total_operating_costs = total_variable_operating_costs +  truck_fixed_o_m  + liquefier_o_m_costs + truck_annual_capital + liquefier_annual_capital
    
    storage_costs = 0.02 #$/kg
    per_unit_costs = total_operating_costs/(delivered_product * annual_plant_cf)+ storage_costs

    fuel_co2_emissions = fuel_usage * h_fuel_ci/ 1000000 * 1000000 / (delivered_product * annual_plant_cf * 365)
    liquefaction_co2_emissions = l_energy_use * elec_CO2_emissions
    total_emissions_per_unit =  fuel_co2_emissions + liquefaction_co2_emissions
    
    return total_operating_costs, per_unit_costs,total_emissions_per_unit

## Pipe Calculations

In [203]:
# Transmission Pipe

def pipe_transmission_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance):

    if delivery_requirement/system_utilization < delivery_requirement:
        mdot_design = delivery_requirement # if I wanted to incorporate leakages
    else:
        mdot_design = delivery_requirement/system_utilization

    transmission_in_meters = transmission_distance * 1000
    dynamic_viscosity = 0.000006 # N-s/m^3
    q_design = mdot_design/ 0.09/ (60*60*24)
    
    temp = 298 # Kelvin
    z = 1
    pipeline_out =  6894757 # Pa (P_out)
    yield_strength = 250 # MPa
    pipe_safety_factor = 0.56
    p_steel = 7801 # kg/m^3
    mdot_normal = delivery_requirement # will be different than the distribution
    # can incorporate leakages into this 
    steel_price = 6.95 #$/kg
    pipe_size_factor = 5 # x mater
    pipe_install = 730000 # $/km
    crf_pipe = 0.15
    crf_compressor = 0.15
    n = 496.031246 #mol/kg
    rgas = 8.314 # J/mol K
    gamma = 1.4
    h2_MW = 0.002016 # kg
    p_atm = 101325 # Pa 
    
    c_1 = ((math.pi**2)* rgas/h2_MW)**0.5* z * temp/ p_atm

    production_p_in = 2000000 # Pa from the assumptions sheet B23
    compressor_efficiency = 1 # could be for MC
    base_compression =  n * 8.314 * 298 * (gamma/(gamma-1))*((pipeline_out/ production_p_in ) ** ((gamma - 1)/ gamma) -1)/1000/compressor_efficiency# kJ/kg

    base_unit_cost = 554 # 4/kW
    base_comp_size = 2000 # #kW
    compressor_factor = 1.1
    comp_exp = 0.8335
    operating_factor = 365
    o_and_m_per_of_cc = 0.05
    H2_LHV = 120000000

    # initializing the storage of the results 
    delivery_cost = []
    energy_loss_perc = []
    co2_emissions = []
    
    for inner_diameter in np.arange(0.017,0.23,0.001): # in meters
        pipe_area = (inner_diameter**2) / 4 * math.pi # m^2
        Re = inner_diameter * ((mdot_design/(60*60*24)) / pipe_area) / dynamic_viscosity # reynold's number
        colebrook_equation = 5.18 * Re ** 0.0909 # colebrook equation that relates the friction factor to the reynolds number and the relative roughness (1/f)^.5
        if (((q_design/((inner_diameter**2.5) * c_1 * colebrook_equation)) ** 2 * (temp * z * transmission_in_meters) + pipeline_out**2)**0.5) > 10000000:
            p_in = 0 # Pa
        else:
            p_in = (((q_design/((inner_diameter**2.5) * c_1 * colebrook_equation)) ** 2 * (temp * z * transmission_in_meters) + pipeline_out**2)**0.5) # Pa


        if p_in == 0:
            delta_p = 0
        else: 
            delta_p = p_in - pipeline_out


        t = p_in * inner_diameter / (2 * yield_strength * 1000000 * pipe_safety_factor - 2 * p_in)
        outer_diameter = inner_diameter + 2 * t # in meters OD in the sheet
        p_design = 2 * yield_strength * t / outer_diameter * 0.8 * 0.7 # in MPa
        w = math.pi * transmission_in_meters * p_steel * t * (inner_diameter + t )  # in kilograms

        actual_re = (inner_diameter * ((mdot_normal/ (60 * 60 * 24))/ (pipe_area)) / dynamic_viscosity)
        actual_colebrook_equation = 5.18 * Re ** 0.0909

        if p_in == 0:
            actual_p_in = 0
        else:
            actual_p_in = (((mdot_normal/0.09/(60*60*24))/(inner_diameter**2.5 * c_1 * actual_colebrook_equation))**2*(temp*z* transmission_in_meters)+ pipeline_out**2)**0.5

        if actual_p_in == 0:
            actual_delta_p = 0
        else:
            actual_delta_p = actual_p_in - pipeline_out

        pipeline_cost = w * steel_price * pipe_size_factor # keeping these values in millions
        pipe_installed_cost = pipe_install * transmission_distance

        annual_pipeline_cost = (pipeline_cost + pipe_installed_cost) * crf_pipe

        if p_in == 0:
            total_pump_work = 0
        else: 
            total_pump_work = n * rgas * temp * (gamma/(gamma - 1)) * ((actual_p_in/ production_p_in)**((gamma-1)/gamma)-1)/1000/compressor_efficiency # in kJ/kg

        design_compressor = (total_pump_work + base_compression) * mdot_design/ (60 * 60 * 24)
        compressor_pipe_p = base_compression #kJ/kg

        if  actual_p_in == 0:
            actual_pump_flow = 0
        else:
            actual_pump_flow = n * rgas * temp * (gamma/(gamma-1)) * ((actual_p_in/ pipeline_out) **((gamma-1)/gamma)-1)/1000/compressor_efficiency

        electricity_compressor = compressor_pipe_p/ 3600
        electricity_pump = actual_pump_flow/ 3600

        cc_compressor_1 = ((base_unit_cost * base_comp_size * (design_compressor * compressor_factor/ base_comp_size) ** comp_exp) * crf_compressor)
        energy_cost_1 = electricity_pump * electricity_cost * mdot_normal * operating_factor
        energy_cost_2 = electricity_compressor * electricity_cost * mdot_normal * operating_factor

        o_and_m_cost = (pipeline_cost + pipe_installed_cost + cc_compressor_1/ crf_compressor) * o_and_m_per_of_cc

        total_annual_cost = energy_cost_1 + energy_cost_2 + cc_compressor_1 + annual_pipeline_cost + o_and_m_cost
        fixed_cost = cc_compressor_1 + annual_pipeline_cost + o_and_m_cost
        variable_cost = energy_cost_1 + energy_cost_2

        del_cost = total_annual_cost/ (delivery_requirement*365) #divided by actual H2 delivered

        total_infrastructure_cost = pipeline_cost + pipe_installed_cost + cc_compressor_1/ crf_compressor
        compressor_cost_per_m = cc_compressor_1 /transmission_in_meters 
        o_and_m_cost_per_m = o_and_m_cost/ transmission_in_meters
        energy_cost_1_per_m = energy_cost_1/ transmission_in_meters
        energy_cost_2_per_m = energy_cost_2/ transmission_in_meters
        total_cost_per_m = total_annual_cost/ transmission_in_meters

        if t > 0:
            
            delivery_cost.append(del_cost)
            energy_loss_perc.append(total_pump_work*1000/H2_LHV*ca_primary_energy)
            co2_emissions.append((electricity_compressor+electricity_pump) * elec_CO2_emissions)
        else: 
            delivery_cost.append(np.nan)            
            energy_loss_perc.append(np.nan)
            co2_emissions.append(np.nan)

    optimized_delivery_cost = np.nanmin(delivery_cost)
    index = np.nanargmin(delivery_cost)
    storage_costs = 0.072 #$/kg of hydrogen
    delivery_cost_with_storage = optimized_delivery_cost + storage_costs
    optimized_energy_loss_perc = energy_loss_perc[index]
    optimized_co2_emissions = co2_emissions[index]

    return delivery_cost_with_storage, optimized_energy_loss_perc, optimized_co2_emissions #delivery_cost_with_storage, optimized_co2_emissions


In [204]:
pipe_transmission_costs(926624.8, 3000, 0.8, 32, 100)

(0.2164436006339709, 0.04698470182796933, 218.01645178083493)

In [254]:
# Transmission Pipe

def pipe_distribution_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance):
    station_count = number_of_stations(delivery_requirement, station_size, system_utilization)
    

    if delivery_requirement/system_utilization < delivery_requirement:
        mdot_design = delivery_requirement*2/(station_count)**0.5 # if I wanted to incorporate leakages
    else:
        mdot_design = delivery_requirement/system_utilization*2/(station_count)**0.5
        
    icr_pipelines_in_meters = city_radius * 2.4 * station_count ** 0.4909 * 1000
    
    dynamic_viscosity = 0.000006 # N-s/m^3
    q_design = mdot_design/ 0.09/ (60*60*24)
    
    temp = 298 # Kelvin
    z = 1
    pipeline_out =  2601000 # Pa (P_out)
    yield_strength = 250 # MPa
    pipe_safety_factor = 0.56
    p_steel = 7801 # kg/m^3
    mdot_normal = delivery_requirement # will be different than the distribution
    # can incorporate leakages into this 
    steel_price = 6.95 #$/kg
    pipe_size_factor = 5 # x mater
    pipe_install = 730000 # $/km
    crf_pipe = 0.15
    crf_compressor = 0.15
    n = 496.031246 #mol/kg
    rgas = 8.314 # J/mol K
    gamma = 1.4
    h2_MW = 0.002016 # kg
    p_atm = 101325 # Pa 
    
    c_1 = ((math.pi**2)* rgas/h2_MW)**0.5* z * temp/ p_atm

    production_p_in = 2600000 # Pa from the assumptions sheet B23
    compressor_efficiency = 1 # could be for MC
    base_compression =  n * 8.314 * 298 * (gamma/(gamma-1))*((pipeline_out/ production_p_in ) ** ((gamma - 1)/ gamma) -1)/1000/compressor_efficiency# kJ/kg

    base_unit_cost = 554 # 4/kW
    base_comp_size = 2000 # #kW
    compressor_factor = 1.1
    comp_exp = 0.8335
    operating_factor = 365
    o_and_m_per_of_cc = 0.05
    H2_LHV = 120000000

    # initializing the storage of the results 
    delivery_cost = []
    energy_loss_perc = []
    co2_emissions = []
    
    for inner_diameter in np.arange(0.017,0.1,0.001): # in meters
        pipe_area = (inner_diameter**2) / 4 * math.pi # m^2
        Re = inner_diameter * ((mdot_design/(60*60*24)) / pipe_area) / dynamic_viscosity # reynold's number
        colebrook_equation = 5.18 * Re ** 0.0909 # colebrook equation that relates the friction factor to the reynolds number and the relative roughness (1/f)^.5
        if (((q_design/((inner_diameter**2.5) * c_1 * colebrook_equation)) ** 2 * (temp * z * icr_pipelines_in_meters) + pipeline_out**2)**0.5) > 10000000:
            p_in = 0 # Pa
        else:
            p_in = (((q_design/((inner_diameter**2.5) * c_1 * colebrook_equation)) ** 2 * (temp * z * icr_pipelines_in_meters) + pipeline_out**2)**0.5) # Pa


        if p_in == 0:
            delta_p = 0
        else: 
            delta_p = p_in - pipeline_out


        t = p_in * inner_diameter / (2 * yield_strength * 1000000 * pipe_safety_factor - 2 * p_in)
        outer_diameter = inner_diameter + 2 * t # in meters OD in the sheet
        p_design = 2 * yield_strength * t / outer_diameter * 0.8 * 0.7 # in MPa
        w = math.pi * icr_pipelines_in_meters * p_steel * t * (inner_diameter + t )  # in kilograms

        actual_re = (inner_diameter * ((mdot_normal/ (60 * 60 * 24))/ (pipe_area)) / dynamic_viscosity)
        actual_colebrook_equation = 5.18 * Re ** 0.0909

        if p_in == 0:
            actual_p_in = 0
        else:
            actual_p_in = (((mdot_normal/0.09/(60*60*24))/(inner_diameter**2.5 * c_1 * actual_colebrook_equation))**2*(temp*z* icr_pipelines_in_meters)+ pipeline_out**2)**0.5

        if actual_p_in == 0:
            actual_delta_p = 0
        else:
            actual_delta_p = actual_p_in - pipeline_out

        pipeline_cost = w * steel_price * pipe_size_factor # keeping these values in millions
        pipe_installed_cost = pipe_install * icr_pipelines_in_meters/ 1000

        annual_pipeline_cost = (pipeline_cost + pipe_installed_cost) * crf_pipe

        if p_in == 0:
            total_pump_work = 0
        else: 
            total_pump_work = n * rgas * temp * (gamma/(gamma - 1)) * ((actual_p_in/ production_p_in)**((gamma-1)/gamma)-1)/1000/compressor_efficiency # in kJ/kg

        design_compressor = (total_pump_work + base_compression) * mdot_design/ (60 * 60 * 24)
        compressor_pipe_p = base_compression #kJ/kg

        if  actual_p_in == 0:
            actual_pump_flow = 0
        else:
            actual_pump_flow = n * rgas * temp * (gamma/(gamma-1)) * ((actual_p_in/ pipeline_out) **((gamma-1)/gamma)-1)/1000/compressor_efficiency

        electricity_compressor = compressor_pipe_p/ 3600
        electricity_pump = actual_pump_flow/ 3600

        cc_compressor_1 = ((base_unit_cost * base_comp_size * (design_compressor * compressor_factor/ base_comp_size) ** comp_exp) * crf_compressor)
        energy_cost_1 = electricity_pump * electricity_cost * mdot_normal * operating_factor
        energy_cost_2 = electricity_compressor * electricity_cost * mdot_normal * operating_factor

        o_and_m_cost = (pipeline_cost + pipe_installed_cost + cc_compressor_1/ crf_compressor) * o_and_m_per_of_cc

        total_annual_cost = energy_cost_1 + energy_cost_2 + cc_compressor_1 + annual_pipeline_cost + o_and_m_cost
        fixed_cost = cc_compressor_1 + annual_pipeline_cost + o_and_m_cost
        variable_cost = energy_cost_1 + energy_cost_2

        del_cost = total_annual_cost/ (delivery_requirement*365) #divided by actual H2 delivered

        total_infrastructure_cost = pipeline_cost + pipe_installed_cost + cc_compressor_1/ crf_compressor
        compressor_cost_per_m = cc_compressor_1 /icr_pipelines_in_meters
        o_and_m_cost_per_m = o_and_m_cost/ icr_pipelines_in_meters
        energy_cost_1_per_m = energy_cost_1/ icr_pipelines_in_meters
        energy_cost_2_per_m = energy_cost_2/ icr_pipelines_in_meters
        total_cost_per_m = total_annual_cost/ icr_pipelines_in_meters

        if t > 0:
            
            delivery_cost.append(del_cost)
            energy_loss_perc.append(total_pump_work*1000/H2_LHV*ca_primary_energy)
            co2_emissions.append((electricity_compressor+electricity_pump) * elec_CO2_emissions)
        else: 
            delivery_cost.append(np.nan)            
            energy_loss_perc.append(np.nan)
            co2_emissions.append(np.nan)

    optimized_delivery_cost = np.nanmin(delivery_cost)
    index = np.nanargmin(delivery_cost)
    #storage_costs = 0.072 #$/kg of hydrogen already added in transmission
    delivery_cost_with_storage = optimized_delivery_cost #+ storage_costs
    optimized_energy_loss_perc = energy_loss_perc[index]
    optimized_co2_emissions = co2_emissions[index]

    return delivery_cost_with_storage, optimized_energy_loss_perc, optimized_co2_emissions #delivery_cost_with_storage, optimized_co2_emissions


## Station Costs

In [273]:
# Does not include leakage rates

def station_costs(station_size, system_utilization):
    if station_size == 1000:
        gas_station_cost = ( 4241366 * 0.084859127 + 4241366 * 0.084859127 * 0.065091824 + 1.79627158733103 * station_size * 365 * system_utilization * electricity_cost)/ station_size/(365*system_utilization)# if you wanted to include leakages /(1-'Costs - Monte Carlo Analysis'!F$12)
        liquid_station_cost = (2677908.69801691* 0.084859127 +2677908.69801691* 0.084859127*0.107193306+0.541339516*station_size*365*system_utilization*electricity_cost)/ station_size/(365*system_utilization)
        pipeline_station_cost = (4241366.138*0.08485913+4241366.138*0.08485913*0.05870373+4.93707297*station_size*365*system_utilization*electricity_cost)/ station_size/(365*system_utilization)

        gas_station_emmissions = 1.79627158733103/ station_size/(365*system_utilization)
        liquid_station_emmissions = 0.541339516/ station_size/(365*system_utilization)
        pipeline_station_emissions = 4.93707297/ station_size/(365*system_utilization)
    if station_size == 2000:
        gas_station_cost = (   6059460  * 0.084859127 + 6059460  * 0.084859127* 0.071325242 + 1.604716869 * station_size * 365 * system_utilization * electricity_cost)/ station_size/(365*system_utilization)## if you wanted to include leakages /(1-'Costs - Monte Carlo Analysis'!F$12)
        liquid_station_cost = (2542674.087* 0.084859127 +2542674.087* 0.084859127*0.121548449+0.540006412*station_size*365*system_utilization*electricity_cost)/ station_size/(365*system_utilization)
        pipeline_station_cost = (6581817.528*0.084859127+6581817.528*0.084859127*0.058132924+4.938022475*station_size*365*system_utilization*electricity_cost)/ station_size/(365*system_utilization)

        gas_station_emmissions = 1.604716869/ station_size/(365*system_utilization)
        liquid_station_emmissions = 0.540006412/ station_size/(365*system_utilization)
        pipeline_station_emissions = 4.938022475/ station_size/(365*system_utilization)
        
    if station_size == 3000:
        gas_station_cost = (7583795   * 0.084859127+ 7583795   * 0.084859127 * 0.074076241 + 1.413162151* station_size * 365 * system_utilization * electricity_cost)/ station_size/(365*system_utilization)# if you wanted to include leakages /(1-'Costs - Monte Carlo Analysis'!F$12)
        liquid_station_cost = (3204929.723* 0.084859127 +3204929.723* 0.084859127*0.135814352+0.541054576*station_size*365*system_utilization*electricity_cost)/ station_size/(365*system_utilization)
        pipeline_station_cost = (8788842.4*0.084859127+8788842.4*0.084859127*0.063667404+4.938662159*station_size*365*system_utilization*electricity_cost)/ station_size/(365*system_utilization)

        gas_station_emmissions = 1.413162151/ station_size/(365*system_utilization)
        liquid_station_emmissions = 0.541054576/ station_size/(365*system_utilization)
        pipeline_station_emissions = 4.938662159/ station_size/(365*system_utilization)
    return gas_station_cost, liquid_station_cost, pipeline_station_cost, gas_station_emmissions, liquid_station_emmissions, pipeline_station_emmissions 

# Total Delivery Costs

In [274]:
# Station Costs
gas_station_cost, liquid_station_cost, pipeline_station_cost, gas_station_emmissions, liquid_station_emmissions, pipeline_station_emmissions  = station_costs(station_size, system_utilization)

# Gas Truck
g_total_operating_costs, g_per_unit_costs,g_total_emissions_per_unit = gas_truck_operating_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance)


# Liquid Truck
l_total_operating_costs, l_per_unit_costs,l_total_emissions_per_unit = liquid_truck_operating_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance)


# Pipelines

t_delivery_cost_with_storage, t_optimized_energy_loss_perc, t_optimized_co2_emissions = pipe_transmission_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance)


d_delivery_cost_with_storage, d_optimized_energy_loss_perc, d_optimized_co2_emissions = pipe_distribution_costs(delivery_requirement, station_size, system_utilization, city_radius, transmission_distance)

NameError: name 'station_size' is not defined